In [ ]:
import openai
import json
from tqdm import tqdm
from datetime import datetime

# CIFAR-10 Class Names
cifar10_classes = [
    "airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"
]

def cupl_func(api_key, api_base, cupl_type="basic", llm_engine="gpt-3.5-turbo", 
              temperature=0.7, max_tokens=100, llm_n=1, save_dir="./"):
    """
    Generates descriptive prompts for CIFAR-10 classes and saves the results in a JSON file.

    Args:
        api_key (str): OpenAI API key.
        api_base (str): OpenAI API base URL.
        cupl_type (str): Type of prompts ("basic" or "full").
        llm_engine (str): Language model engine to use.
        temperature (float): Sampling temperature for OpenAI responses.
        max_tokens (int): Maximum number of tokens for each response.
        llm_n (int): Number of completions per prompt.
        save_dir (str): Directory to save the resulting JSON file.

    Returns:
        tuple: A dictionary of responses and the file path of the saved JSON file.
    """
    # Initialize OpenAI client
    client = openai.OpenAI(api_key=api_key, base_url=api_base)

    # JSON filename
    json_name = f"{save_dir}cifar10.json"

    all_responses = {}
    vowel_list = ['A', 'E', 'I', 'O', 'U', 'a', 'e', 'i', 'o', 'u']

    # Iterate over each CIFAR-10 class
    for category in tqdm(cifar10_classes):
        print(f"Processing category: {category}")
        article = "an" if category[0] in vowel_list else "a"
        
        prompts = [
            f"Describe what {article} {category} looks like",
            f"How can you identify {article} {category}?",
            f"What does {article} {category} look like?"
        ]
        
        # Add additional prompts if "full" type is selected
        if cupl_type == "full":
            prompts.append(f"Describe an image from the internet of {article} {category}")
            prompts.append(f"A caption of an image of {article} {category}:")

        all_result = []
        for curr_prompt in prompts:
            # Generate responses using OpenAI's chat completion
            response = client.chat.completions.create(
                model=llm_engine,
                messages=[{"role": "user", "content": curr_prompt}],
                stream=False,
                temperature=temperature,
                max_tokens=max_tokens,
                n=llm_n
            )
            # Collect and process each response
            for r in response.choices:
                result = r.message.content
                all_result.append(result.replace("\n\n", "") + ".")

        all_responses[category] = all_result

    # Save the responses to a JSON file
    with open(json_name, 'w') as f:
        json.dump(all_responses, f, indent=4)
    print(f"Successfully saved the results in: {json_name}")

    return all_responses, json_name

# Example usage (API key and base URL must be set by the user)
# Note: Replace these placeholders with valid API key and base URL.
api_key = "your_api_key_here"  # Set your OpenAI API key
api_base = "your_api_base_url_here"  # Set your OpenAI API base URL

# Call the function
cupl_func(api_key, api_base)
